In [2]:
# --- Cell 1: configuration + imports ---

import os, time, math, datetime as dt
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("POLYGON_API_KEY")
if not API_KEY:
    raise ValueError("❌ POLYGON_API_KEY not found in .env")

BASE = "https://api.massive.com"
NY_TZ = "America/New_York"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
os.makedirs("data", exist_ok=True)

In [ ]:
# --- Cell 2: Black-Scholes helpers ---

def norm_cdf(x):
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def norm_pdf(x):
    return 1.0 / math.sqrt(2*math.pi) * math.exp(-0.5 * x * x)

def bs_call_price(S, K, T, r, sigma):
    if T <= 0:
        return max(S - K, 0.0)
    if sigma <= 0:
        return max(S - K * math.exp(-r*T), 0.0)
    d1 = (math.log(S/K) + (r + 0.5*sigma*sigma)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return S * norm_cdf(d1) - K * math.exp(-r*T) * norm_cdf(d2)

def bs_put_price(S, K, T, r, sigma):
    if T <= 0:
        return max(K - S, 0.0)
    if sigma <= 0:
        return max(K * math.exp(-r*T) - S, 0.0)
    d1 = (math.log(S/K) + (r + 0.5*sigma*sigma)*T) / (sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    return K * math.exp(-r*T) * norm_cdf(-d2) - S * norm_cdf(-d1)

def bs_vega(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return None
    d1 = (math.log(S/K) + (r + 0.5*sigma*sigma)*T) / (sigma*math.sqrt(T))
    return S * norm_pdf(d1) * math.sqrt(T)

def implied_vol_call(S, K, T, r, price, max_iter=60, tol=1e-6):
    intrinsic = max(S - K * math.exp(-r*T), 0.0)
    if price is None or price <= intrinsic + 1e-4 or S <= 0 or K <= 0 or T <= 0:
        return None
    low, high = 1e-4, 3.0
    for _ in range(max_iter):
        mid = 0.5*(low+high)
        diff = bs_call_price(S, K, T, r, mid) - price
        if abs(diff) < tol:
            return mid
        if diff > 0:
            high = mid
        else:
            low = mid
    return 0.5*(low+high)

def implied_vol_put(S, K, T, r, price, max_iter=60, tol=1e-6):
    intrinsic = max(K * math.exp(-r*T) - S, 0.0)
    if price is None or price <= intrinsic + 1e-4 or S <= 0 or K <= 0 or T <= 0:
        return None
    low, high = 1e-4, 3.0
    for _ in range(max_iter):
        mid = 0.5*(low+high)
        diff = bs_put_price(S, K, T, r, mid) - price
        if abs(diff) < tol:
            return mid
        if diff > 0:
            high = mid
        else:
            low = mid
    return 0.5*(low+high)

# --- Extra BS helpers: d1/d2 + full Greeks ---

def bs_d1_d2(S, K, T, r, sigma):
    """
    Common d1, d2 helper. Assumes S, K, T, sigma > 0.
    """
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return None, None
    d1 = (math.log(S/K) + (r + 0.5 * sigma * sigma) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    return d1, d2

def bs_call_greeks(S, K, T, r, sigma):
    """
    Return call price, delta, gamma, vega, theta (theta per day).
    Vega is per 1 vol point (i.e. 0.01 change in sigma).
    """
    d1, d2 = bs_d1_d2(S, K, T, r, sigma)
    if d1 is None:
        return None, None, None, None, None

    price = bs_call_price(S, K, T, r, sigma)
    delta = norm_cdf(d1)
    gamma = norm_pdf(d1) / (S * sigma * math.sqrt(T))

    v_raw = bs_vega(S, K, T, r, sigma)
    vega = v_raw / 100.0 if v_raw is not None else None

    theta_annual = (
        - (S * norm_pdf(d1) * sigma) / (2 * math.sqrt(T))
        - r * K * math.exp(-r * T) * norm_cdf(d2)
    )
    theta = theta_annual / 365.0

    return price, delta, gamma, vega, theta

def bs_put_greeks(S, K, T, r, sigma):
    """
    Return put price, delta, gamma, vega, theta (theta per day).
    Vega is per 1 vol point (i.e. 0.01 change in sigma).
    """
    d1, d2 = bs_d1_d2(S, K, T, r, sigma)
    if d1 is None:
        return None, None, None, None, None

    price = bs_put_price(S, K, T, r, sigma)
    delta = norm_cdf(d1) - 1.0
    gamma = norm_pdf(d1) / (S * sigma * math.sqrt(T))

    v_raw = bs_vega(S, K, T, r, sigma)
    vega = v_raw / 100.0 if v_raw is not None else None

    theta_annual = (
        - (S * norm_pdf(d1) * sigma) / (2 * math.sqrt(T))
        + r * K * math.exp(-r * T) * norm_cdf(-d2)
    )
    theta = theta_annual / 365.0

    return price, delta, gamma, vega, theta

In [ ]:
# --- Cell 3: API helpers ---

def fetch_underlying_daily(sym, start_date, end_date):
    """
    Pull 1-day OHLC for the underlying for the whole window.
    """
    url = f"{BASE}/v2/aggs/ticker/{sym}/range/1/day/{start_date}/{end_date}"
    params = {"adjusted": "true", "sort": "asc", "limit": 50000}
    r = requests.get(url, params=params, headers=HEADERS, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"{sym} daily {start_date}→{end_date}: {r.text[:200]}")
    j = r.json()
    rows = j.get("results", [])
    if not rows:
        return pd.DataFrame(columns=["date_ny","S_open","S_close"])
    df = pd.DataFrame(rows).rename(columns={"t":"ts","o":"open","c":"close"})
    df["ts_utc"] = pd.to_datetime(df["ts"], unit="ms", utc=True)
    df["ts_ny"]  = df["ts_utc"].dt.tz_convert(NY_TZ)
    df["date_ny"] = df["ts_ny"].dt.date
    df = df.sort_values("date_ny")
    return df[["date_ny","open","close"]].rename(columns={"open":"S_open","close":"S_close"})

def list_contracts_asof(sym, asof_date):
    """
    Get all option contracts for this underlying as of a certain date.
    """
    url = f"{BASE}/v3/reference/options/contracts"
    params = {
        "underlying_ticker": sym,
        "include_expired": "true",
        "as_of": pd.Timestamp(asof_date).strftime("%Y-%m-%d"),
        "limit": 1000,
        "order": "asc",
    }
    r = requests.get(url, params=params, headers=HEADERS, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"{sym} contracts {asof_date}: {r.text[:200]}")
    return pd.DataFrame(r.json().get("results", []))

def pick_nearest_expiry_atm(df_contracts, spot, asof_date,
                            min_days_ahead=3, max_days_ahead=30):
    """
    From today's contract list, pick the nearest expiry (min_days_ahead–max_days_ahead)
    and then pick ATM call + ATM put.
    """
    if df_contracts is None or df_contracts.empty:
        return None, None, None
    keep = df_contracts[["ticker","contract_type","strike_price","expiration_date"]].dropna()
    if keep.empty:
        return None, None, None
    keep["strike_price"] = keep["strike_price"].astype(float)
    keep["expiration_date"] = pd.to_datetime(keep["expiration_date"]).dt.date

    as_of = pd.Timestamp(asof_date).date()
    keep["days_to_exp"] = (keep["expiration_date"] - as_of).apply(lambda x: x.days)

    fwd = keep[(keep["days_to_exp"] >= min_days_ahead) & (keep["days_to_exp"] <= max_days_ahead)].copy()
    if fwd.empty:
        return None, None, None

    expiry = fwd.sort_values(["days_to_exp","expiration_date"]).iloc[0]["expiration_date"]
    near = fwd[fwd["expiration_date"] == expiry]

    calls = near[near["contract_type"]=="call"].copy()
    puts  = near[near["contract_type"]=="put"].copy()
    if calls.empty or puts.empty:
        return None, None, None

    calls["dist"] = (calls["strike_price"] - spot).abs()
    puts["dist"]  = (puts["strike_price"]  - spot).abs()

    call_info = calls.sort_values("dist").iloc[0].to_dict()
    put_info  = puts.sort_values("dist").iloc[0].to_dict()
    return expiry, call_info, put_info

def pick_expiry_strike_band(
    df_contracts,
    spot,
    asof_date,
    min_days_ahead=3,
    max_days_ahead=30,
    n_each_side=10,
):
    """
    Choose a single expiry (nearest within [min_days_ahead, max_days_ahead]),
    then return a band of call + put strikes around ATM:
    - for each side (calls/puts), sort by strike
    - find the strike closest to spot
    - take ~n_each_side strikes below and above that
    """
    if df_contracts is None or df_contracts.empty:
        return None, None, None

    keep = df_contracts[["ticker", "contract_type", "strike_price", "expiration_date"]].dropna()
    if keep.empty:
        return None, None, None

    keep["strike_price"] = keep["strike_price"].astype(float)
    keep["expiration_date"] = pd.to_datetime(keep["expiration_date"]).dt.date

    as_of = pd.Timestamp(asof_date).date()
    keep["days_to_exp"] = (keep["expiration_date"] - as_of).apply(lambda x: x.days)

    # restrict to desired DTE window
    fwd = keep[
        (keep["days_to_exp"] >= min_days_ahead)
        & (keep["days_to_exp"] <= max_days_ahead)
    ].copy()
    if fwd.empty:
        return None, None, None

    # pick nearest expiry in time
    expiry = fwd.sort_values(["days_to_exp", "expiration_date"]).iloc[0]["expiration_date"]
    near = fwd[fwd["expiration_date"] == expiry].copy()

    calls = near[near["contract_type"] == "call"].copy()
    puts  = near[near["contract_type"] == "put"].copy()
    if calls.empty or puts.empty:
        return None, None, None

    def _band_around_atm(df_side):
        # sort by strike
        df_side = df_side.sort_values("strike_price").reset_index(drop=True)
        # index of strike closest to spot
        atm_idx = (df_side["strike_price"] - spot).abs().idxmin()
        low = max(atm_idx - n_each_side, 0)
        high = min(atm_idx + n_each_side, len(df_side) - 1)
        return df_side.iloc[low:high+1].copy()

    calls_band = _band_around_atm(calls)
    puts_band  = _band_around_atm(puts)

    return expiry, calls_band, puts_band

def fetch_option_bar(opt_ticker, day):
    """
    Try daily first; if missing, fallback to minute and aggregate volume.
    """
    start_ymd = pd.Timestamp(day).strftime("%Y-%m-%d")
    end_ymd   = pd.Timestamp(day + dt.timedelta(days=1)).strftime("%Y-%m-%d")

    # 1) daily
    url_day = f"{BASE}/v2/aggs/ticker/{opt_ticker}/range/1/day/{start_ymd}/{end_ymd}"
    r = requests.get(url_day, headers=HEADERS, timeout=20)
    if r.status_code == 200:
        rows = r.json().get("results", [])
        if rows:
            x = rows[0]
            return {"volume": x.get("v", 0), "close": x.get("c", None)}

    # 2) fallback: minute
    url_min = f"{BASE}/v2/aggs/ticker/{opt_ticker}/range/1/minute/{start_ymd}/{end_ymd}"
    params = {"adjusted":"true","sort":"asc","limit":50000}
    r = requests.get(url_min, params=params, headers=HEADERS, timeout=30)
    rows = r.json().get("results", [])
    if not rows:
        return {"volume": 0, "close": None}
    vol = sum(row.get("v", 0) for row in rows)
    last_close = rows[-1].get("c", None)
    return {"volume": vol, "close": last_close}

In [5]:
# --- Cell 4: original daily builder (still available if you want it) ---

def build_daily_options(sym, start_date, end_date):
    """
    Original daily version:
    - daily underlying
    - contracts every day
    - ATM 1-45 DTE
    - option daily → minute fallback
    - compute IV + vega
    """
    print(f"\n[{sym}] DAILY build {start_date} → {end_date}")
    px = fetch_underlying_daily(sym, start_date, end_date)
    if px.empty:
        raise RuntimeError(f"No underlying daily for {sym}")

    rows = []

    for _, row in px.iterrows():
        day     = row["date_ny"]
        S_open  = float(row["S_open"])
        S_close = float(row["S_close"])

        # 1) contracts for this exact day
        try:
            dfc = list_contracts_asof(sym, day)
        except Exception as e:
            print(f"[{sym}] contracts fail {day}: {e}")
            continue

        expiry, call_info, put_info = pick_nearest_expiry_atm(
            dfc, S_close, day, min_days_ahead=1, max_days_ahead=45
        )
        if expiry is None:
            print(f"[{sym}] no suitable ATM on {day}")
            continue

        # 2) option bars
        call_bar = fetch_option_bar(call_info["ticker"], day)
        put_bar  = fetch_option_bar(put_info["ticker"],  day)

        call_vol   = call_bar["volume"]
        call_price = call_bar["close"]
        put_vol    = put_bar["volume"]
        put_price  = put_bar["close"]
        total_vol  = (call_vol or 0) + (put_vol or 0)

        # 3) time to expiry
        asof = pd.Timestamp(day).date()
        dte  = max((expiry - asof).days, 1)
        T    = dte / 365.0
        r    = 0.0

        # 4) IV + vega
        call_iv = call_vega = None
        if call_price is not None and call_price > 0.05:
            Kc = float(call_info["strike_price"])
            call_iv = implied_vol_call(S_close, Kc, T, r, call_price)
            if call_iv is None:
                call_iv = 0.20
            v = bs_vega(S_close, Kc, T, r, call_iv)
            call_vega = v/100 if v is not None else None

        put_iv = put_vega = None
        if put_price is not None and put_price > 0.05:
            Kp = float(put_info["strike_price"])
            put_iv = implied_vol_put(S_close, Kp, T, r, put_price)
            if put_iv is None:
                put_iv = 0.20
            v = bs_vega(S_close, Kp, T, r, put_iv)
            put_vega = v/100 if v is not None else None

        ivs = [x for x in [call_iv, put_iv] if x is not None]
        avg_iv = sum(ivs)/len(ivs) if ivs else None

        vegas = [x for x in [call_vega, put_vega] if x is not None]
        avg_vega = sum(vegas)/len(vegas) if vegas else None

        rows.append({
            "date_ny": day,
            "S_open": S_open,
            "S_close": S_close,
            "call_ticker": call_info["ticker"],
            "put_ticker":  put_info["ticker"],
            "strike_call": float(call_info["strike_price"]),
            "strike_put":  float(put_info["strike_price"]),
            "expiry": pd.Timestamp(expiry),
            "days_to_exp": dte,
            "call_vol": call_vol,
            "put_vol": put_vol,
            "total_vol": total_vol,
            "call_price": call_price,
            "put_price": put_price,
            "call_iv": call_iv,
            "put_iv": put_iv,
            "avg_iv": avg_iv,
            "call_vega": call_vega,
            "put_vega": put_vega,
            "avg_vega": avg_vega,
        })

        print(f"[{sym}] {day} | Cvol={call_vol} Pvol={put_vol} | Cpx={call_price} Ppx={put_price}")

        # tiny pause
        time.sleep(0.02)

    df_out = pd.DataFrame(rows).sort_values("date_ny")
    return df_out

In [ ]:
# --- Cell 5: NVDA snapshot using latest available data (as live as possible) ---

def get_latest_underlying(sym, max_days_back=5):
    """
    Try to get the most recent intraday (1-minute) data for `sym`.
    - First tries today, then yesterday, etc. up to `max_days_back` days.
    - Uses the LAST minute bar of the first day that has data.
    - If no intraday data is found, falls back to daily bars.
    Returns: (asof_date, last_close_price)
    """
    today = dt.date.today()

    # 1) Try intraday minute data, walking backwards day by day
    for offset in range(0, max_days_back):
        day = today - dt.timedelta(days=offset)
        start_ymd = day.strftime("%Y-%m-%d")
        end_ymd   = (day + dt.timedelta(days=1)).strftime("%Y-%m-%d")

        url = f"{BASE}/v2/aggs/ticker/{sym}/range/1/minute/{start_ymd}/{end_ymd}"
        params = {"adjusted": "true", "sort": "asc", "limit": 50000}
        r = requests.get(url, params=params, headers=HEADERS, timeout=30)

        if r.status_code == 200:
            rows = r.json().get("results", [])
            if rows:
                last_close = rows[-1].get("c", None)
                if last_close is not None:
                    print(f"[{sym}] Using latest intraday from {day}: close={last_close}")
                    return day, float(last_close)

    # 2) Fallback to daily data if no intraday found (e.g. long holiday)
    for offset in range(1, max_days_back + 6):
        day = today - dt.timedelta(days=offset)
        day_str = day.strftime("%Y-%m-%d")
        try:
            px = fetch_underlying_daily(sym, day_str, day_str)
        except Exception:
            continue
        if not px.empty:
            S_close = float(px.iloc[-1]["S_close"])
            print(f"[{sym}] Fallback daily close from {day}: close={S_close}")
            return day, S_close

    raise RuntimeError(f"[{sym}] Could not find recent underlying data within {max_days_back} days.")

def pick_expiry_strike_band(
    df_contracts,
    spot,
    asof_date,
    min_days_ahead=3,
    max_days_ahead=30,
    n_each_side=10,
):
    """
    Choose 1 expiry (nearest within [min_days_ahead, max_days_ahead]),
    then return a band of call + put strikes around ATM:
    - find ATM strike (closest to spot)
    - take n_each_side strikes below and above ATM
    """
    if df_contracts is None or df_contracts.empty:
        return None, None, None

    keep = df_contracts[["ticker", "contract_type", "strike_price", "expiration_date"]].dropna()
    if keep.empty:
        return None, None, None

    keep["strike_price"] = keep["strike_price"].astype(float)
    keep["expiration_date"] = pd.to_datetime(keep["expiration_date"]).dt.date

    as_of = pd.Timestamp(asof_date).date()
    keep["days_to_exp"] = (keep["expiration_date"] - as_of).apply(lambda x: x.days)

    # Filter by DTE window
    fwd = keep[
        (keep["days_to_exp"] >= min_days_ahead) &
        (keep["days_to_exp"] <= max_days_ahead)
    ].copy()
    if fwd.empty:
        return None, None, None

    # Pick nearest expiry
    expiry = fwd.sort_values(["days_to_exp", "expiration_date"]).iloc[0]["expiration_date"]
    near = fwd[fwd["expiration_date"] == expiry].copy()

    calls = near[near["contract_type"] == "call"].copy()
    puts  = near[near["contract_type"] == "put"].copy()
    if calls.empty or puts.empty:
        return None, None, None

    def _band(df_side):
        df_side = df_side.sort_values("strike_price").reset_index(drop=True)
        atm_idx = (df_side["strike_price"] - spot).abs().idxmin()
        low = max(atm_idx - n_each_side, 0)
        high = min(atm_idx + n_each_side, len(df_side) - 1)
        return df_side.iloc[low:high+1].copy()

    calls_band = _band(calls)
    puts_band  = _band(puts)

    return expiry, calls_band, puts_band

def build_nvda_greeks(
    asof_date=None,
    min_days_ahead=3,
    max_days_ahead=30,
    n_each_side=10,
    r=0.0,
):
    """
    FINAL SNAPSHOT FUNCTION (LIVE VERSION):
    - If asof_date is None:
        -> Pull the *latest* intraday NVDA data Polygon/Massive has.
    - Else:
        -> Use that specific date (daily close).
    - Select one expiry (nearest in DTE window).
    - Take ATM ± n_each_side strikes for calls AND puts.
    - Fetch option prices for each contract (same as-of date).
    - Compute IV + delta, gamma, vega, theta for each contract.

    Returns: DataFrame (one row per contract).
    """
    sym = "NVDA"

    # 1) Determine as-of date and underlying price
    if asof_date is None:
        asof, S_close = get_latest_underlying(sym)
    else:
        asof = pd.Timestamp(asof_date).date()
        day_str = asof.strftime("%Y-%m-%d")
        px = fetch_underlying_daily(sym, day_str, day_str)
        if px.empty:
            raise RuntimeError(f"No NVDA underlying daily for {asof}")
        S_close = float(px.iloc[-1]["S_close"])

    print(f"\n[{sym}] Snapshot for {asof} | S_close={S_close}")

    # 2) Contracts as of this date
    dfc = list_contracts_asof(sym, asof)
    if dfc is None or dfc.empty:
        raise RuntimeError(f"No NVDA contracts returned for {asof}")

    expiry, calls_band, puts_band = pick_expiry_strike_band(
        dfc,
        spot=S_close,
        asof_date=asof,
        min_days_ahead=min_days_ahead,
        max_days_ahead=max_days_ahead,
        n_each_side=n_each_side,
    )
    if expiry is None:
        raise RuntimeError(f"No suitable NVDA band found for {asof}")

    print(
        f"[{sym}] Chosen expiry {expiry} | "
        f"Calls={len(calls_band)} | Puts={len(puts_band)}"
    )

    # 3) Time to expiry
    dte = max((expiry - asof).days, 1)
    T = dte / 365.0

    rows = []

    # 4) Process CALLS
    for _, opt in calls_band.iterrows():
        opt_ticker = opt["ticker"]
        K = float(opt["strike_price"])

        bar = fetch_option_bar(opt_ticker, asof)
        price = bar["close"]
        if price is None or price <= 0.05:
            continue

        iv = implied_vol_call(S_close, K, T, r, price)
        if iv is None:
            iv = 0.20

        _, delta, gamma, vega, theta = bs_call_greeks(S=S_close, K=K, T=T, r=r, sigma=iv)

        rows.append({
            "underlying": sym,
            "date_ny": asof,
            "option_type": "call",
            "opt_ticker": opt_ticker,
            "strike": K,
            "S_close": S_close,
            "expiry": expiry,
            "days_to_exp": dte,
            "option_price": price,
            "implied_vol": iv,
            "delta": delta,
            "gamma": gamma,
            "vega": vega,
            "theta": theta,
        })

    # 5) Process PUTS
    for _, opt in puts_band.iterrows():
        opt_ticker = opt["ticker"]
        K = float(opt["strike_price"])

        bar = fetch_option_bar(opt_ticker, asof)
        price = bar["close"]
        if price is None or price <= 0.05:
            continue

        iv = implied_vol_put(S_close, K, T, r, price)
        if iv is None:
            iv = 0.20

        _, delta, gamma, vega, theta = bs_put_greeks(S=S_close, K=K, T=T, r=r, sigma=iv)

        rows.append({
            "underlying": sym,
            "date_ny": asof,
            "option_type": "put",
            "opt_ticker": opt_ticker,
            "strike": K,
            "S_close": S_close,
            "expiry": expiry,
            "days_to_exp": dte,
            "option_price": price,
            "implied_vol": iv,
            "delta": delta,
            "gamma": gamma,
            "vega": vega,
            "theta": theta,
        })

    df_snapshot = pd.DataFrame(rows)
    df_snapshot = df_snapshot.sort_values(["option_type", "strike"])
    return df_snapshot


In [7]:
if __name__ == "__main__":
    # Latest available NVDA snapshot (live-ish)
    df_nvda = build_nvda_greeks(n_each_side=10)
    print(df_nvda)

    # Save to CSV
    out_path = f"data/NVDA_latest_band_{len(df_nvda)}rows.csv"
    df_nvda.to_csv(out_path, index=False)
    print(f"\nSaved to: {out_path}")

[NVDA] Using latest intraday from 2025-11-18: close=180.7197

[NVDA] Snapshot for 2025-11-18 | S_close=180.7197
[NVDA] Chosen expiry 2025-11-21 | Calls=21 | Puts=21
   underlying     date_ny option_type             opt_ticker  strike  \
0        NVDA  2025-11-18        call  O:NVDA251121C00155000   155.0   
1        NVDA  2025-11-18        call  O:NVDA251121C00157500   157.5   
2        NVDA  2025-11-18        call  O:NVDA251121C00160000   160.0   
3        NVDA  2025-11-18        call  O:NVDA251121C00162500   162.5   
4        NVDA  2025-11-18        call  O:NVDA251121C00165000   165.0   
5        NVDA  2025-11-18        call  O:NVDA251121C00167500   167.5   
6        NVDA  2025-11-18        call  O:NVDA251121C00170000   170.0   
7        NVDA  2025-11-18        call  O:NVDA251121C00172500   172.5   
8        NVDA  2025-11-18        call  O:NVDA251121C00175000   175.0   
9        NVDA  2025-11-18        call  O:NVDA251121C00177500   177.5   
10       NVDA  2025-11-18        call  O:NV